In [1]:
import json

qasper = json.load(open("./data/qasper-test-v0.3.json",'r'))

In [ ]:
from transformers import AutoTokenizer
from transformers import T5ForConditionalGeneration

import torch

# model = T5ForConditionalGeneration.from_pretrained("castorini/monot5-3b-msmarco-10k", torch_dtype=torch.float16) #t5 3b
model = T5ForConditionalGeneration.from_pretrained('castorini/monot5-base-msmarco-10k')

tokenizer = AutoTokenizer.from_pretrained("t5-3b")

device = torch.device("cuda")
model.to(device)

In [ ]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5

reranker = MonoT5(model=model)

In [ ]:
from tqdm import tqdm
import uuid

c = 1
qas = []
for k in qasper:
    print("\n{0} of {1}\n".format(c, len(qasper.keys())))
    item = qasper[k]
    paragraphs =  [p  for section in item['full_text'] for p in  section["paragraphs"]]
    texts = [Text(p, {'docid': i,"title":p}, 0) for i,p in enumerate(paragraphs)]
    for qa in tqdm(item['qas']):
        query = Query(qa['question'])
        reranked = reranker.rerank(query, texts)
        reranked.sort(key=lambda x: x.score, reverse=True)
        if  "answers" in qa:
            del qa['answers']
        qa['documents'] = [t.metadata for t in reranked]
        qas.append(qa)
    c += 1

In [ ]:
json.dump(qas, open("qasper_reranked_section_title.json","w"))